# Merge camera labels to GPS and tractor logs

  * The goal for this notebook is to merge camera labels to GPS and tractor logs from 07/15/2019.
  * `label.h5` and `gps.h5` has the same data rate (1 Hz).
  * `label.h5` and `tra.h5` has different data rates:
    * Merge using `merge_asof`
  * The outputs should have two dataframes:
    * Labeled GPS dataframe
    * Labeled tractor log dataframe

In [ ]:
import pandas as pd
import numpy as np

First we need to load in labels, GPS, and tractor log data.

In [ ]:
labels = pd.read_hdf('./labels.h5')
gps = pd.read_hdf('/home/yang/research/dock/explicator/gps-07152019.h5')
tra = pd.read_hdf('/home/yang/research/dock/explicator/tra-07152019.h5')

The `tra` and `gps` dataframes do not have `ts` as the index. To simplify merging, we will set these indices first.

In [ ]:
tra.index = pd.to_datetime(tra['ts'], unit='s')
gps.index = pd.to_datetime(gps['ts'], unit='s')

In [ ]:
gps_df = gps.sort_index().drop(columns=['ts'])
gps_df.head(10)

In [ ]:
tra_df = tra.sort_index().drop(columns=['ts'])
tra_df.head(10)

Now we can merge the `gps_df` and `tra_df` with `labels`.

In [ ]:
gps_labeled_df = pd.merge(gps_df, labels, how='inner', left_index=True, right_index=True)

In [ ]:
gps_labeled_df.head(10)

In [ ]:
tra_labeled_df = pd.merge_asof(tra_df, labels, left_index=True, right_index=True, \
                               direction='nearest', tolerance=pd.Timedelta('1s'))

In [ ]:
tra_labeled_df = tra_labeled_df[tra_labeled_df['left'].notna()]

In [ ]:
tra_labeled_df.head(10)

Now we can save the labeled dataframes.

In [ ]:
gps_labeled_df.to_hdf('./gps_labeled.h5', key='gps_label', complevel=5)
tra_labeled_df.to_hdf('./tra_labeled.h5', key='tra_label', complevel=5)